In [1]:
import sys
sys.path.append('./')
import torch
import numpy as np
from src.dmtet.dmtet_test import DMTetGeometry
from src.mesh import TetMesh
from src.deform import Deform
from src.visualize import viewer

res = 64
num_iter = 1000
DMTet = DMTetGeometry(res=res, iter=num_iter)
verts, tets = DMTet.getMesh()
tet_mesh = TetMesh(verts, tets) 
# viewer(verts.cpu().detach(), tets.cpu().detach(), draw_tet=True).show()

# 这里假设gt audio是全为0的声音
gt_audio = torch.zeros((1, 1000)).cuda()
loss_func = torch.nn.MSELoss()

optimizer = torch.optim.Adam(DMTet.parameters(), lr=0.03)

for i in range(100):
    audio_loss, reg_loss = DMTet.tick(gt_audio, loss_func, i)
    print("iter:{}, audio_loss: {}, reg_loss: {}".format(i, audio_loss, reg_loss))

    total_loss = audio_loss + reg_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

verts, tets = DMTet.getMesh()


viewer(verts.cpu().detach(), tets.cpu().detach(), draw_tet=True).show()


Box(children=(Box(children=(FigureWidget({
    'data': [{'i': array([  2,   8,   8, ..., 103, 103,  86]),
    …